In [1]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [4]:
edges = cg.read_chunk_edges([cg.get_chunk_id(layer=2,x=50,y=100,z=0)])
edges = edges["between"].get_pairs()
edges

array([[75582903194484819, 75582834475009044],
       [75582903194484867, 75582834475009018],
       [75582903194484873, 75582834475009193],
       [75582903194484912, 75512534450307165],
       [75582903194484963, 75582834475009168],
       [75582903194485200, 75512534450307435],
       [75582903194485330, 75512534450307707],
       [75582903194485481, 75512534450307707],
       [75582903194485506, 75582971913961513],
       [75582903194485507, 75582971913961542],
       [75582903194485544, 75582971913961501],
       [75582903194485666, 75582971913961503],
       [75582903194485697, 75582971913961503],
       [75582903194485720, 75512534450307957],
       [75582903194485720, 75582971913961693],
       [75582903194485753, 75582971913961503]], dtype=uint64)

In [2]:
# cg.get_parents(np.unique(edges))
cg.get_parents([75582903194484819, 75582903194485753])

array([147640497232412675, 147640497232412673], dtype=uint64)

In [2]:
from pychunkedgraph.graph.edits_v2 import add_edge_v2
add_edge_v2(cg, edge=[75582903194484819, 75582903194485753])

{147640497232412673: {2: array([[75582903194485753, 75582971913961501],
       [75582903194485544, 75582971913961501],
       [75582903194485544, 75582971913961488],
       [75582903194485720, 75582971913961693],
       [75582903194485720, 75582971913961473],
       [75582903194485767, 75582971913961693]], dtype=uint64), 3: array([[75582903194485200, 75512534450307435],
       [75582903194485200, 75512534450307545],
       [75582903194485330, 75512534450307653],
       [75582903194485481, 75512534450307737],
       [75582903194485490, 75512534450307910],
       [75582903194485528, 75512534450307957],
       [75582903194485720, 75512534450307957],
       [75582903194485720, 75512534450308044],
       [75582903194485163, 75512534450307435],
       [75582903194485077, 75512534450307285],
       [75582903194484912, 75512534450307165],
       [75582903194485109, 75512534450307419]], dtype=uint64), 4: array([[75582903194484746, 75582834475009094],
       [75582903194484873, 75582834475009193

ValueError: Coordinate is out of range             layer: 7 bits/dim 5.             [25, 50, 0]; max = 32.